In [ ]:
# Add the parent directory to the path
import sys, os
sys.path.insert(0, os.path.abspath("../.."))

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

import torch
import random
import pyepo
from tabulate import tabulate

from src.data.config import HP

from src.solvers.spnia_asym import AsymmetricSPNI
from src.models.ShortestPathGrb import shortestPathGrb
from src.models.ShortestPathGrid import ShortestPathGrid
from src.solvers.BendersDecomposition import BendersDecomposition
from models.CalibrationTrainer import CalibratorTrainer

from scripts.compare_po_spo import compare_po_spo
from scripts.setup import gen_data, gen_train_data, setup_po_model, setup_hybrid_spo_model

## Set Parameters

In [ ]:
# Initialize the configuration class
cfg = HP()

# Set the random seed for reproducibility
np.random.seed(cfg.get("random_seed"))
random.seed(cfg.get("random_seed"))
torch.manual_seed(cfg.get("random_seed"))
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(cfg.get("random_seed"))


In [ ]:
cfg.set("num_scenarios", 10)

## Generate Network Data
Includes the network graph, a shortest path opt_model, and training data.

In [ ]:
# Define a graph with appropriate dimensions and an opt_model 
# for solving the shortest path problem on the graph
m,n = cfg.get("grid_size")
graph = ShortestPathGrid(m,n)
opt_model = shortestPathGrb(graph)

# Generate normalized training and testing data
training_data, testing_data, normalization_constant = gen_train_data(cfg, opt_model)

## Predict-then-Optimize

In [ ]:
po_model = setup_po_model(
    cfg,
    graph,
    opt_model,
    training_data,
    versatile=True
)

## Smart Predict-then-Optimize with OptNet

In [ ]:
spo_model = setup_hybrid_spo_model(
    cfg,
    graph,
    opt_model,
    training_data,
    versatile=True
)

In [ ]:
# print(f"Old SPO predictor regret:                  {pyepo.metric.regret(spo_model, opt_model, training_data['val_loader'])}")
# print(f"log_s parameter before Calibrator fitting: {spo_model.log_s.item()}") 
# calibration_trainer = CalibratorTrainer(spo_model)
# calibration_trainer.align_scale_ols(training_data['train_loader'])
# print(f"log_s parameter after Calibrator fitting:  {spo_model.log_s.item()}")
# print(f"New SPO predictor regret:                  {pyepo.metric.regret(spo_model, opt_model, training_data['val_loader'])}")

## Compare PO and SPO
Comparison of performance for solving shortest path problems

In [ ]:
true_objs, po_objs, spo_objs = compare_po_spo(cfg, opt_model, po_model, spo_model, testing_data)

In [ ]:
po_val = [(po - true) / true * 100 for po, true in zip(po_objs, true_objs)]
spo_val = [(spo - true) / true * 100 for spo, true in zip(spo_objs, true_objs)]

# plt.scatter(np.ones_like(po_objs), po_val)
# plt.scatter(np.ones_like(spo_objs) + 1, spo_val)

fig, ax = plt.subplots(figsize=(6,4))

# Two groups: PO and SPO
_ = ax.boxplot([po_val, spo_val], tick_labels=['PO', 'SPO'],
           showmeans=True, meanline=True)   # optional styling
ax.set_ylabel("Cost increase [%]")
ax.set_ylim(0,250)
ax.grid(True)
# plt.legend(['PO', 'SPO'], location='north')

po_mean  = np.mean(po_val)
spo_mean = np.mean(spo_val)

# std devs
po_std_samp = np.std(po_val, ddof=1)
spo_std_samp = np.std(spo_val, ddof=1)
mean_improvement = (po_mean - spo_mean) * 100 / po_mean
std_improvement = (po_std_samp - spo_std_samp) * 100 / po_std_samp

rows = [
    ["Mean",         f"{po_mean:.2f}%",  f"{spo_mean:.2f}%", f"{mean_improvement:.2f}%"],
    ["Std (sample)", f"{po_std_samp:.2f}%", f"{spo_std_samp:.2f}%", f"{std_improvement:.2f}%"],
]
print(tabulate(rows, headers=["Metric", "PO (%)", "SPO (%)", "SPO improvement (%)"], tablefmt="github"))

## Compare Different Interdictors
We train different prediction models for the interdictors and compare their performance

In [ ]:
interdictions = gen_data(cfg, seed=cfg.get("intd_seed"), normalization_constant=normalization_constant)

In [ ]:
# Comparison of different means of costs to show similar 
print(f"Mean value comparison:")
print(f"----------------------")
print(f"Test:  {testing_data['costs'].mean():.7f}")
print(f"Train: {training_data['train_loader'].dataset.costs.mean():.7f}")
print(f"Intd:  {interdictions['costs'].mean():.7f}")
print(f"PO:    {po_model(torch.tensor(testing_data['feats'], dtype=torch.float32)).mean().item():.7f}")
print(f"SPO+:  {spo_model(torch.tensor(testing_data['feats'], dtype=torch.float32)).mean().item():.7f}")

print(f"\nStd value comparison:")
print(f"----------------------")
print(f"Test:  {testing_data['costs'].std():.7f}")
print(f"Train: {training_data['train_loader'].dataset.costs.std():.7f}")
print(f"Intd:  {interdictions['costs'].std():.7f}")
print(f"PO:    {po_model(torch.tensor(testing_data['feats'], dtype=torch.float32)).std().item():.7f}")
print(f"SPO+:  {spo_model(torch.tensor(testing_data['feats'], dtype=torch.float32)).std().item():.7f}")

### Naive Interdictor
This interdictor assumes that the evader has full information.

In [ ]:
def compare_sym_intd(cfg, po_model, spo_model, test_data, interdictions, normalization_constant, idx = None):
    """
    Compare the performance of the PO and SPO models using symmetric shortest path interdiction.
    This function simulates the interdiction process and evaluates the objective values.
    """

    # Get the number of simulation data samples
    num_test_samples = cfg.get("num_test_samples")

    # Prepare lists to store results
    true_objs = []
    po_objs = []
    spo_objs = []

    # Print that the simulation is starting
    print(f"Running simulation with {num_test_samples} samples...")

    # Iterate through each data sample
    for i in range(num_test_samples) if idx is None else [idx]:
        # Store values for the current sample
        feature = test_data["feats"][i]
        cost = test_data["costs"][i] * normalization_constant
        interdiction = interdictions["costs"][i] * normalization_constant
        m, n = cfg.get("grid_size")
        
        # Update opt_model
        true_graph = ShortestPathGrid(m, n, cost=cost)
        opt_model = shortestPathGrb(true_graph)

        # Update the estimated costs
        po_cost = po_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy() * normalization_constant
        spo_cost = spo_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy() * normalization_constant

        # Solutions without information asymmetry
        interdictor_I = BendersDecomposition(opt_model, 
                                             k=cfg.get("budget"), 
                                             interdiction_cost=interdiction, 
                                             max_cnt=cfg.get("benders_max_count"), 
                                             eps=cfg.get("benders_eps"))
        x_intd, _, _ = interdictor_I.solve(versatile=False if idx is None else True)

        # True shortest path after interdiction
        opt_model.setObj(cost + x_intd * interdiction)
        y_true, _ = opt_model.solve()

        # PO estimated shortest path after interdiction
        opt_model.setObj(po_cost + x_intd * interdiction)
        y_po, _ = opt_model.solve()

        # SPO estimated shortest path after interdiction
        opt_model.setObj(spo_cost + x_intd * interdiction)
        y_spo, _ = opt_model.solve()

        # Store the results
        true_objs.append(true_graph(y_true, interdictions=x_intd * interdiction))
        po_objs.append(true_graph(y_po, interdictions=x_intd * interdiction))
        spo_objs.append(true_graph(y_spo, interdictions=x_intd * interdiction))

        step = max(1, num_test_samples // 20)  # ~every 5% (safe for small N)
        if (i % step == 0) or (i == num_test_samples - 1):
            if i == num_test_samples - 1:
                done, pct = 20, 100                      # final tick
            else:
                done = (i * 20) // num_test_samples      # 0..20 “=”
                pct  = done * 5                          # 0,5,10,...,95

            sys.stdout.write('\r[%-20s] %3d%%' % ('=' * done, pct))
            sys.stdout.flush()
            if i == num_test_samples - 1:
                sys.stdout.write('\n')


    # Evaluate performance
    return {
        "true_objective": np.array(true_objs),
        "po_objective": np.array(po_objs),
        "spo_objective": np.array(spo_objs),
    }

In [ ]:
all_pred_sym_intd = compare_sym_intd(cfg, po_model, spo_model, testing_data, interdictions, normalization_constant)
# compare_sym_intd(cfg, po_model, spo_model, testing_data, interdictions, normalization_constant, idx=0)

### Knowledgeable Interdictor

In [ ]:
def compare_asym_intd(cfg, test_data, interdictions, normalization_constant, pred_model = None):
    """
    Compare the performance of the predicted model with the true model
    using asymmetric shortest path interdiction.
    """ 

    # Get the number of simulation data samples
    num_test_samples = cfg.get("num_test_samples")

    # Print that the simulation is starting
    print(f"Running simulation with {num_test_samples} samples...")

    # Prepare lists to store results
    true_objs = []
    est_objs = []

    # Iterate through each data sample
    for i in range(num_test_samples):
        # Store values for the current sample
        cost = test_data["costs"][i] * normalization_constant
        interdiction = interdictions["costs"][i] * normalization_constant
        m, n = cfg.get("grid_size")

        # Compute estimated cost if an estimator is provided. Otherwise use the true costs
        if pred_model is not None:
            feature = test_data["feats"][i]
            pred_cost = pred_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy() * normalization_constant
        else:
            pred_cost = cost
        
        # Create graph with true costs and opt_model for estimated shortest path
        true_graph = ShortestPathGrid(m, n, cost=cost)
        est_opt_model = shortestPathGrb(true_graph)
        est_opt_model.setObj(pred_cost)

        # Solutions with information asymmetry
        asym_interdictor = AsymmetricSPNI(
            true_graph, 
            budget=cfg.get("budget"), 
            true_costs=cost, 
            true_delays=interdiction, 
            est_costs=pred_cost, 
            est_delays=interdiction, 
            lsd=cfg.get("lsd")
        )
        x_intd, _ = asym_interdictor.solve()

        # # True shortest path after interdiction
        # opt_model.setObj(cost + x_intd * interdiction)
        # y_true, _ = opt_model.solve()

        # Estimated shortest path after interdiction
        est_opt_model.setObj(pred_cost + x_intd * interdiction)
        y_est, _ = est_opt_model.solve()

        # Store the results
        # true_objs.append(true_graph(y_true, interdictions=x_intd * interdiction))
        est_objs.append(true_graph(y_est, interdictions=x_intd * interdiction))

        # Print progress
        step = max(1, num_test_samples // 20)  # ~every 5% (safe for small N)
        if (i % step == 0) or (i == num_test_samples - 1):
            if i == num_test_samples - 1:
                done, pct = 20, 100                      # final tick
            else:
                done = (i * 20) // num_test_samples      # 0..20 “=”
                pct  = done * 5                          # 0,5,10,...,95

            sys.stdout.write('\r[%-20s] %3d%%' % ('=' * done, pct))
            sys.stdout.flush()
            if i == num_test_samples - 1:
                sys.stdout.write('\n')

    # Evaluate performance
    # return {
    #     "true_objective": np.array(true_objs),
    #     "estimated_objective": np.array(est_objs),
    # }
    return np.array(est_objs)

In [ ]:
def compare_wrong_asym_intd(cfg, test_data, interdictions, normalization_constant, true_model, false_model):
    """
    Evader uses true model and interdictor uses asymmetric SPNI assuming false model.
    """ 

    # Get the number of simulation data samples
    num_test_samples = cfg.get("num_test_samples")

    # Print that the simulation is starting
    print(f"Running simulation with {num_test_samples} samples...")

    # Prepare lists to store results
    true_objs = []
    est_objs = []

    # Iterate through each data sample
    for i in range(num_test_samples):
        # Store values for the current sample
        cost = test_data["costs"][i] * normalization_constant
        interdiction = interdictions["costs"][i] * normalization_constant
        m, n = cfg.get("grid_size")

        # Compute estimated cost if an estimator is provided. Otherwise use the true costs
        feature = test_data["feats"][i]
        true_pred_cost = true_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy() * normalization_constant
        false_pred_cost = false_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy() * normalization_constant

        # Create graph with true costs and opt_model for estimated shortest path
        true_graph = ShortestPathGrid(m, n, cost=cost)
        est_opt_model = shortestPathGrb(true_graph)

        # Solutions with information asymmetry
        asym_interdictor = AsymmetricSPNI(
            true_graph, 
            budget=cfg.get("budget"), 
            true_costs=cost, 
            true_delays=interdiction, 
            est_costs=false_pred_cost, 
            est_delays=interdiction, 
            lsd=cfg.get("lsd")
        )
        x_intd, _ = asym_interdictor.solve()

        # # True shortest path after interdiction
        # opt_model.setObj(cost + x_intd * interdiction)
        # y_true, _ = opt_model.solve()

        # Estimated shortest path after interdiction
        est_opt_model.setObj(true_pred_cost + x_intd * interdiction)
        y_est, _ = est_opt_model.solve()

        # Store the results
        # true_objs.append(true_graph(y_true, interdictions=x_intd * interdiction))
        est_objs.append(true_graph(y_est, interdictions=x_intd * interdiction))

        # Print progress
        step = max(1, num_test_samples // 20)  # ~every 5% (safe for small N)
        if (i % step == 0) or (i == num_test_samples - 1):
            if i == num_test_samples - 1:
                done, pct = 20, 100                      # final tick
            else:
                done = (i * 20) // num_test_samples      # 0..20 “=”
                pct  = done * 5                          # 0,5,10,...,95

            sys.stdout.write('\r[%-20s] %3d%%' % ('=' * done, pct))
            sys.stdout.flush()
            if i == num_test_samples - 1:
                sys.stdout.write('\n')

    # Evaluate performance
    # return {
    #     "true_objective": np.array(true_objs),
    #     "estimated_objective": np.array(est_objs),
    # }
    return np.array(est_objs)

In [ ]:
no_pred_asym_intd = compare_asym_intd(cfg, testing_data, interdictions, normalization_constant)

In [ ]:
po_pred_asym_intd_I = compare_asym_intd(cfg, testing_data, interdictions, normalization_constant, po_model)

In [ ]:
spo_pred_asym_intd_I = compare_asym_intd(cfg, testing_data, interdictions, normalization_constant, spo_model)

In [ ]:
true_spo_false_po_asym_intd = compare_wrong_asym_intd(cfg, testing_data, interdictions, normalization_constant, spo_model, po_model)

In [ ]:
true_po_false_spo_asym_intd = compare_wrong_asym_intd(cfg, testing_data, interdictions, normalization_constant, po_model, spo_model)

In [ ]:
eps = 1e-5  # Define a small epsilon for numerical stability
print(f"Instances where Sym is better than Asym: {sum([1 if sym - asym > eps else 0 for sym, asym in zip(all_pred_sym_intd['spo_objective'], spo_pred_asym_intd_I)]) / len(po_objs)}")
# print(f"Instances where SPO is better than PO: {sum([1 if po - spo > eps else 0 for po, spo in zip(all_pred_sym_intd['spo_objective'], spo_pred_asym_intd)]) / len(po_objs)}")
# print(f"Sanity check (equal performance):      {sum([1 if abs(po - spo) < eps else 0 for po, spo in zip(all_pred_sym_intd['spo_objective'], spo_pred_asym_intd)]) / len(po_objs)}")
# Interesting result: The percentage of PO better than SPO is almost equal to SPO better than PO.

In [ ]:
# [(asym - sym).item() for sym, asym in zip(all_pred_sym_intd['spo_objective'], spo_pred_asym_intd)]
# print(all_pred_sym_intd['true_objective'][0])
# print(no_pred_asym_intd[0])
# testing_data['costs'].std()
# sym is better than asym in 28% of instances without calibration

In [ ]:
# Precentage improvements, read as: 
# <prediction type>_<original interdiction>_<changed interdiction> 
# -> percentage is improvement from the original interdiction to the changed interdiction
# po_no_sym = (po_pred_sym_intd-po_pred_no_intd)/po_pred_no_intd
# po_no_asym = (po_pred_asym_intd-po_pred_no_intd)/po_pred_no_intd
# spo_no_sym = (spo_pred_sym_intd-spo_pred_no_intd)/spo_pred_no_intd
# spo_no_asym = (spo_pred_asym_intd-spo_pred_no_intd)/spo_pred_no_intd

# Percentage improvements, read as:
# <original prediction type>_<changed prediction type>_<interdiction type>
# -> percentage is improvement from the original prediction type to the changed prediction type
# po_spo_no = (spo_pred_no_intd-po_pred_no_intd)/po_pred_no_intd
# po_spo_sym = (spo_pred_sym_intd-po_pred_sym_intd)/po_pred_sym_intd
# po_spo_asym = (spo_pred_asym_intd-po_pred_asym_intd)/po_pred_asym_intd

# Prepare no-interdiction results for printing
true_mean = np.array(true_objs).mean() * normalization_constant
po_mean = np.array(po_objs).mean() * normalization_constant
spo_mean = np.array(spo_objs).mean() * normalization_constant

# Print the results in a table format
rows = [
    ["Oracle", f"{true_mean:.2f}", f"{all_pred_sym_intd['true_objective'].mean():.2f}", f"{no_pred_asym_intd.mean():.2f}", f"N/A"],
    ["PO", f"{po_mean:.2f} ",  f"{all_pred_sym_intd['po_objective'].mean():.2f} ({np.nan:.2f}%)", f"{po_pred_asym_intd_I.mean():.2f} ({np.nan:.2f}%)", f"{true_po_false_spo_asym_intd.mean():.2f} (Evader=PO, Intd=SPO)"],
    ["SPO", f"{spo_mean:.2f} ", f"{all_pred_sym_intd['spo_objective'].mean():.2f} ({np.nan:.2f}%)", f"{spo_pred_asym_intd_I.mean():.2f} ({np.nan:.2f}%)", f"{true_spo_false_po_asym_intd.mean():.2f} (Evader=SPO, Intd=PO)"],
    ["Improvement", f"{np.nan:.2f}%", f"{np.nan:.2f}%", f"{np.nan:.2f}%", f"{np.nan:.2f}%"]
]
print(tabulate(rows, headers=["Predictor", "No Interdictor", "Sym. Interdictor", "Asym. Interdictor", "Asym. Intd. (Row is True Evader)"], tablefmt="github"))

# Boxplot of the true objectives for both interdiction methods
fig, ax = plt.subplots(figsize=(6,4))
_ = ax.boxplot([all_pred_sym_intd["po_objective"], all_pred_sym_intd["spo_objective"], po_pred_asym_intd_I, spo_pred_asym_intd_I],
           tick_labels=['Symmetric PO', 'Symmetric SPO', 'Asymmetric PO', 'Asymmetric SPO'],
           showmeans=True, meanline=True)
ax.set_ylabel("Interdicted Shortest Path Length")
ax.grid(True)

## Comparison with Single Instances

In [ ]:
idx = 1

# Create figure
fig, ax = plt.subplots(1,3,figsize=(18,6))
print("Uninterdicted Shortest Paths")
print("----------------------------")

# Store constants that will be reused
cost = testing_data['costs'][idx] * normalization_constant
feature = testing_data['feats'][idx]
interdiction_I = interdictions['costs'][idx] * normalization_constant

# Create the true graph and models
true_graph = ShortestPathGrid(m, n, cost=cost)
true_opt_model = shortestPathGrb(true_graph)

# Create the PO and SPO graph & models
po_cost = po_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy() * normalization_constant
spo_cost = spo_model(torch.tensor(feature, dtype=torch.float32)).detach().numpy() * normalization_constant
po_opt_model = shortestPathGrb(true_graph)
spo_opt_model = shortestPathGrb(true_graph)
po_opt_model.setObj(po_cost)
spo_opt_model.setObj(spo_cost)

# Solve & visualize the shortest paths for ...
# ... the true model
path, obj = true_opt_model.solve()
true_opt_model.visualize(ax=ax[0], colored_edges=path, title="Oracle")
true_pred_no_intd_I = true_graph(path)
print(f"True cost: {true_pred_no_intd_I:.2f}")

# ... the PO model
path, obj = po_opt_model.solve()
po_opt_model.visualize(ax=ax[1], colored_edges=path, title="PO")
po_pred_no_intd_I = true_graph(path)
print(f"PO cost: {po_pred_no_intd_I:.2f}")

# ... the SPO model
path, obj = spo_opt_model.solve()
spo_opt_model.visualize(ax=ax[2], colored_edges=path, title="SPO")
spo_pred_no_intd_I = true_graph(path)
print(f"SPO cost: {spo_pred_no_intd_I:.2f}")

fig.tight_layout()

In [ ]:
po_cost

In [ ]:
# Solutions wit information symmetry
interdictor_I = BendersDecomposition(true_opt_model, k=B, interdiction_cost=interdiction_I, max_cnt=cfg.get("benders_max_count"), eps=cfg.get("benders_eps"))
x_sym, y_sym, obj_sym = interdictor_I.solve()

# Solutions with information asymmetry for Oracle (Should be identical to symmetric solution)
asym_true_interdictor = AsymmetricSPNI(true_graph, budget=B, true_costs=cost, true_delays=interdiction_I, est_costs=cost, est_delays=interdiction_I, lsd=1e-5)
x_asym_true, obj_asym_true = asym_true_interdictor.solve()

# Solutions with information asymmetry for SPO
asym_spo_interdictor = AsymmetricSPNI(true_graph, budget=B, true_costs=cost, true_delays=interdiction_I, est_costs=spo_cost, est_delays=interdiction_I, lsd=1e-5)
x_asym_spo, obj_asym_spo = asym_spo_interdictor.solve()

# Solutions with information asymmetry for PO
asym_po_interdictor = AsymmetricSPNI(true_graph, budget=B, true_costs=cost, true_delays=interdiction_I, est_costs=po_cost, est_delays=interdiction_I, lsd=1e-5)
x_asym_po, obj_asym_po = asym_po_interdictor.solve()

In [ ]:
# Create figure
fig, ax2 = plt.subplots(1,3,figsize=(18,6))
print("Symmetric Interdiction")
print("----------------------")

# Solve & visualize the shortest paths for ...
# ... the oracle (same results for sym/asym interdiction)
true_opt_model.setObj(cost + x_sym * interdiction_I)
path, obj = true_opt_model.solve()
true_opt_model.visualize(ax=ax2[0], colored_edges=path, dashed_edges=x_sym, title="Oracle")
true_pred_sym_intd_I = true_graph(path, interdictions=x_sym * interdiction_I)
print(f"Oracle cost: {true_pred_sym_intd_I:.2f}")

# ... the PO model & sym interdiction
po_opt_model.setObj(po_cost + x_sym * interdiction_I)
path, obj = po_opt_model.solve()
po_opt_model.visualize(ax=ax2[1], colored_edges=path, dashed_edges=x_sym, title="PO")
po_pred_sym_intd_I = true_graph(path, interdictions=x_sym * interdiction_I)
print(f"PO cost: {po_pred_sym_intd_I:.2f}")

# ... the SPO model & sym interdiction
spo_opt_model.setObj(spo_cost + x_sym * interdiction_I)
path, obj = spo_opt_model.solve()
spo_opt_model.visualize(ax=ax2[2], colored_edges=path, dashed_edges=x_sym, title="SPO")
spo_pred_sym_intd_I = true_graph(path, interdictions=x_sym * interdiction_I)
print(f"SPO cost: {spo_pred_sym_intd_I:.2f}")

fig.tight_layout()

In [ ]:
# Create figure
fig, ax3 = plt.subplots(1,3,figsize=(18,6))
print("Asymmetric Interdiction")
print("-----------------------")

# Solve & visualize the shortest paths for ...
# ... the oracle & asym interdiction
true_opt_model.setObj(cost + x_asym_true * interdiction_I)
path, obj = true_opt_model.solve()
true_opt_model.visualize(ax=ax3[0], colored_edges=path, dashed_edges=x_asym_true, title="Oracle")
true_pred_asym_intd_I = true_graph(path, interdictions=x_asym_true * interdiction_I)
print(f"Oracle cost: {true_pred_asym_intd_I:.2f}")

# ... the PO model & asym interdiction
po_opt_model.setObj(po_cost + x_asym_po * interdiction_I)
path, obj = po_opt_model.solve()
po_opt_model.visualize(ax=ax3[1], colored_edges=path, dashed_edges=x_asym_po, title="PO")
po_pred_asym_intd_I = true_graph(path, interdictions=x_asym_po * interdiction_I)
print(f"PO cost: {po_pred_asym_intd_I:.2f}")

# ... the SPO model & asym interdiction
spo_opt_model.setObj(spo_cost + x_asym_spo * interdiction_I)
path, obj = spo_opt_model.solve()
spo_opt_model.visualize(ax=ax3[2], colored_edges=path, dashed_edges=x_asym_spo, title="SPO")
spo_pred_asym_intd_I = true_graph(path, interdictions=x_asym_spo * interdiction_I)
print(f"SPO cost: {spo_pred_asym_intd_I:.2f}")

fig.tight_layout()

In [ ]:
# Precentage improvements, read as: 
# <prediction type>_<original interdiction>_<changed interdiction> 
# -> percentage is improvement from the original interdiction to the changed interdiction
po_no_sym = (po_pred_sym_intd_I-po_pred_no_intd_I)/po_pred_no_intd_I
po_no_asym = (po_pred_asym_intd_I-po_pred_no_intd_I)/po_pred_no_intd_I
spo_no_sym = (spo_pred_sym_intd_I-spo_pred_no_intd_I)/spo_pred_no_intd_I
spo_no_asym = (spo_pred_asym_intd_I-spo_pred_no_intd_I)/spo_pred_no_intd_I

# Percentage improvements, read as:
# <original prediction type>_<changed prediction type>_<interdiction type>
# -> percentage is improvement from the original prediction type to the changed prediction type
po_spo_no = (spo_pred_no_intd_I-po_pred_no_intd_I)/po_pred_no_intd_I
po_spo_sym = (spo_pred_sym_intd_I-po_pred_sym_intd_I)/po_pred_sym_intd_I
po_spo_asym = (spo_pred_asym_intd_I-po_pred_asym_intd_I)/po_pred_asym_intd_I

# Print the results in a table format
rows = [
    ["Oracle", f"{true_pred_no_intd_I:.2f}", f"{true_pred_sym_intd_I:.2f}", f"{true_pred_asym_intd_I:.2f}"],
    ["PO",         f"{po_pred_no_intd_I:.2f} ",  f"{po_pred_sym_intd_I:.2f} ({po_no_sym*100:.2f}%)", f"{po_pred_asym_intd_I:.2f} ({po_no_asym*100:.2f}%)"],
    ["SPO", f"{spo_pred_no_intd_I:.2f} ", f"{spo_pred_sym_intd_I:.2f} ({spo_no_sym*100:.2f}%)", f"{spo_pred_asym_intd_I:.2f} ({spo_no_asym*100:.2f}%)"],
    ["Improvement", f"{po_spo_no*100:.2f}%", f"{po_spo_sym*100:.2f}%", f"{po_spo_asym*100:.2f}%"]
]
print(tabulate(rows, headers=["Predictor", "No Interdictor", "Sym. Interdictor", "Asym. Interdictor"], tablefmt="github"))

In [ ]:
# Explanation of the table:
# - Interdictors want to maximize the cost, so they try to achieve high percentages
# - Evaders (Predictors) want to minimize the shortest path, so they try to achieve low percentages

# Observations from above table:
# - We can clearly see that the SPO model outperforms the PO model in all cases
# - However, employing a symetric interdiction strategy decreses the cost for the PO model -> small performance difference between PO and SPO

# Possibly iteresting takeaways:
# - Can we show that it doesn't matter much if the evader uses PO or SPO if we deploy bender's (kinda obvious bc bender's is the safe variant tho)